In [ ]:
import netgen.occ as occ
from ngsolve import *
from ngsolve.internal import *
import numpy as np
import matplotlib.pyplot as plt
from ngsolve.krylovspace import CGSolver
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
import time as t

geo=occ.Cylinder(p=occ.Pnt(0,0,0),d=occ.Dir((0,0,1)),r=1,h=1)-occ.Cylinder(p=occ.Pnt(0,0,0),d=occ.Dir((0,0,1)),r=0.5,h=1)

geo.faces[0].name = "Zp"
geo.faces[1].name = "bot"
geo.faces[2].name = "up"
geo.faces[3].name = "Zm"

geo = occ.OCCGeometry(geo)
simplemesh = geo.GenerateMesh(maxh=0.1)
simplemesh= Mesh(simplemesh)
Draw(simplemesh)

In [ ]:
fesVectorPotential = HCurl(simplemesh, dirichlet = "Zp|Zm|up|bot")
TotalCurrent = 1
    
normal = specialcf.normal(simplemesh.dim)
gfuDirichlet = GridFunction(fesVectorPotential)
g = CF([(0,0,TotalCurrent) if bc=="Zm" else (0,0,0) for bc in simplemesh.GetBoundaries()])
gfuDirichlet.Set(g,BND)

Draw(gfuDirichlet, simplemesh, vectors = { "grid_size":200},clipping = {"x" : 0, "y" : -1, "z" : 0, "dist" : 0})

In [ ]:
    # Weak form
T = fesVectorPotential.TrialFunction()
Tstar = fesVectorPotential.TestFunction()
Kt = BilinearForm(curl(Tstar)*curl(T)*dx)
    
    # Assembly
#with TaskManager():
Kt.Assemble()

r = - Kt.mat * gfuDirichlet.vec

    # Solving
maxres = 1e-8
maxit = 1000

Tsol = GridFunction(fesVectorPotential)
print("Solving...")
#with TaskManager():
iterativeSolver = CGSolver(Kt.mat, freedofs=fesVectorPotential.FreeDofs(), tol  = maxres,  maxiter  = maxit)
Tsol.vec.data = iterativeSolver * r + gfuDirichlet.vec

if len(iterativeSolver.residuals)==maxit: print("... Failure!")
else: print("... Success!")
print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")

Draw(curl(Tsol), simplemesh,vectors = { "grid_size":50},clipping = {"x" : 0, "y" : 0, "z" : 1, "dist" : 0})